# **Feature Extraction**

In [ ]:
from keras.models import load_model
import os
from keras.models import Model
from PIL import Image
from keras.preprocessing import image
import numpy as np
import cv2
import pandas as pd
from keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading the pre-trained model
model = load_model('/content/drive/My Drive/master_dataset/Gmodel.h5')

In [ ]:
layer_output = model.get_layer('flatten').output

In [ ]:
intermediate_model = Model(inputs = model.input, outputs = layer_output)

In [ ]:
input_image = image.load_img('/content/drive/My Drive/master_dataset/test/class0/Im181.jpg', target_size = (256,256))
input_image = image.img_to_array(input_image)
input_image = np.expand_dims(input_image, axis = 0)
intermediate_output = intermediate_model.predict(input_image)

1/1 [==============================] - 0s 82ms/step


In [ ]:
df = pd.DataFrame(intermediate_output)
df.to_csv('/content/drive/My Drive/feature.csv', index = False)

In [ ]:
# Specify the layer from which you want to extract features
feature_extraction_model = Model(inputs=model.input, outputs=model.get_layer('dense').output)

In [ ]:
# Function to extract features from a directory
def extract_features_from_directory(directory):
    features_list = []
    labels = []
    for class_folder in os.listdir(directory):
        class_folder_path = os.path.join(directory, class_folder)
        for filename in os.listdir(class_folder_path):
            img_path = os.path.join(class_folder_path, filename)
            img = image.load_img(img_path, target_size=(256, 256))
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            features = feature_extraction_model.predict(img)
            features_list.append(features.flatten())  # Flatten the features
            labels.append(class_folder)  # Assuming the folder name represents the class (e.g., class0, class1)
    return np.array(features_list), np.array(labels)

In [ ]:
# Extract features and labels from train and test directories
train_features, train_labels = extract_features_from_directory('/content/drive/My Drive/master_dataset/train')
test_features, test_labels = extract_features_from_directory('/content/drive/My Drive/master_dataset/test')

In [ ]:
# Create pandas dataframes for train and test features
train_df = pd.DataFrame(train_features)
train_df['label'] = train_labels
test_df = pd.DataFrame(test_features)
test_df['label'] = test_labels

In [ ]:
# Save train and test features and labels to CSV files
train_df.to_csv('/content/drive/My Drive/master_dataset/train_features.csv', index=False)
test_df.to_csv('/content/drive/My Drive/master_dataset/test_features.csv', index=False)